In [1]:
import os
from getpass import getpass
# os.environ["HF_API_TOKEN"] = getpass("Enter Hugging Face token: ")

In [2]:
import os
os.environ['OPENAI_API_KEY']='sk-111111111111111111111111111111111111111111111111'
os.environ['OPENAI_API_BASE']='http://127.0.0.1:5000/v1'

In [3]:
import requests
model_info_url = 'http://127.0.0.1:5000/v1/internal/model/info'
resp = requests.get(model_info_url)
model = resp.json()['model_name']

print(model)

Mixtral-8x7B-instruct-exl2-6.0bpw


In [4]:
from haystack.components.generators import HuggingFaceTGIGenerator, OpenAIGenerator

# generator = HuggingFaceTGIGenerator("mistralai/Mixtral-8x7B-Instruct-v0.1")
# generator.warm_up()
generator = OpenAIGenerator(api_base_url=os.getenv('OPENAI_API_BASE'))   

In [5]:
result = generator.run("What's Natural Language Processing?", generation_kwargs={"max_tokens": 350})
print(result["replies"][0])

Natural Language Processing, often abbreviated as NLP, is a subfield of artificial intelligence (AI) that focuses on the interaction between computers and human (natural) languages. The ultimate objective of NLP is to read, decipher, understand, and make sense of the human language in a valuable way. 

By using various computational techniques, including machine learning and deep learning, NLP attempts to understand the structure of language, the meaning of words, the context of words and sentences, and how to use this information to perform tasks like translation, sentiment analysis, speech recognition, and text summarization.

Practical applications of NLP are everywhere, such as search engines, voice-activated assistants (Siri, Alexa, Google Assistant), grammar checkers, email filters, and machine translators (Google Translate), to name a few.


In [6]:
from haystack.components.fetchers.link_content import LinkContentFetcher
from haystack.components.converters import HTMLToDocument
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.rankers import TransformersSimilarityRanker
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack import Pipeline
from haystack.utils import ComponentDevice

In [7]:
fetcher = LinkContentFetcher()
converter = HTMLToDocument()
document_splitter = DocumentSplitter(split_by="word", split_length=50)
similarity_ranker = TransformersSimilarityRanker(top_k=3, device=ComponentDevice.from_str("cuda:1"))

In [8]:
prompt_template = """
According to these documents:

{% for doc in documents %}
  {{ doc.content }}
{% endfor %}

Answer the given question: {{question}}
Answer:
"""

In [9]:
prompt_builder = PromptBuilder(template=prompt_template)

In [10]:
pipeline = Pipeline()
pipeline.add_component("fetcher", fetcher)
pipeline.add_component("converter", converter)
pipeline.add_component("splitter", document_splitter)
pipeline.add_component("ranker", similarity_ranker)
pipeline.add_component("prompt_builder", prompt_builder)
pipeline.add_component("llm", generator)

In [11]:
pipeline.connect("fetcher.streams", "converter.sources")
pipeline.connect("converter.documents", "splitter.documents")
pipeline.connect("splitter.documents", "ranker.documents")
pipeline.connect("ranker.documents", "prompt_builder.documents")
pipeline.connect("prompt_builder.prompt", "llm.prompt")

In [12]:
question = "What do graphs have to do with Haystack?"

In [13]:
result = pipeline.run({"prompt_builder": {"question": question},
                   "ranker": {"query": question},
                   "fetcher": {"urls": ["https://haystack.deepset.ai/blog/introducing-haystack-2-beta-and-advent"]},
                    "llm":{"generation_kwargs": {"max_tokens": 350}}})

In [14]:
print(result['llm']['replies'][0])

In the context of Haystack, a text ranking and question-answering framework, graphs are used to represent pipelines of components that process and rank documents. In Haystack 1.x, these pipelines are implemented as directed acyclic graphs (DAGs), which are graphs that do not contain cycles and where edges have a direction. This means that in Haystack 1.x, the components in the pipeline are arranged in a linear fashion, with data flowing from one component to the next.

In Haystack 2.0, the use of graphs to represent pipelines is being extended to allow for more complex arrangements of components. Specifically, Haystack 2.0 will allow for the creation of pipelines that can branch out, join, and cycle back to another component. This is achieved by removing the requirement that the graphs be acyclical, resulting in directed graphs (also known as digraphs) rather than DAGs. This allows for the creation of pipelines that can retry, loop back, and potentially even run in parallel.
